In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import random
import copy
import logging
import logging.config

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
tf.compat.v1.disable_eager_execution()

In [2]:
class data_generation():
    def __init__(self, type):
        print('init')
        self.data_type = type
        self.dataset = './data/' + self.data_type + '.csv'

        self.train_users = []
        self.train_sessions = []  # 当前的session
        self.train_items = []  # 随机采样得到的positive
        self.train_neg_items = []  # 随机采样得到的negative
        self.train_pre_sessions = []  # 之前的session集合

        self.test_users = []
        self.test_candidate_items = []
        self.test_sessions = []
        self.test_pre_sessions = []
        self.test_real_items = []

        self.neg_number = 5
        self.user_number = 0
        self.item_number = 0
        self.train_batch_id = 0
        self.test_batch_id = 0
        self.records_number = 0

    def gen_train_test_data(self):
        self.data = pd.read_csv(self.dataset, names=['user', 'sessions'], dtype='str')
        is_first_line = 1
        for line in self.data.values:
            if is_first_line:
                self.user_number = int(line[0])
                self.item_number = int(line[1])
                print("user_number:",self.user_number)
                print("item_number:",self.item_number)
                self.user_purchased_item = dict()  # 保存每个用户购买记录，可用于train时负采样和test时剔除已打分商品
                is_first_line = 0
            else:
                user_id = int(line[0])
                sessions = [i for i in line[1].split('@')]
                size = len(sessions)
                the_first_session = [int(i) for i in sessions[0].split(':')]
                self.train_pre_sessions.append(the_first_session)
                tmp = copy.deepcopy(the_first_session)
                self.user_purchased_item[user_id] = tmp # 儲存用戶第一個購物籃項目串列
                for j in range(1, size - 1):
                    # 每个用户的每个session在train_users中都对应着其user_id
                    self.train_users.append(user_id)
                    current_session = [int(it) for it in sessions[j].split(':')]
                    neg = self.gen_neg(user_id)
                    self.train_neg_items.append(neg)
                    # 将当前session加入到用户购买的记录当中
                    # 之所以放在这个位置，是因为在选择测试item时，需要将session中的一个item移除、
                    # 如果放在后面操作，当前session中其实是少了一个用来做当前session进行预测的item
                    if j != 1:
                        tmp = copy.deepcopy(self.user_purchased_item[user_id])
                        self.train_pre_sessions.append(tmp)
                    self.user_purchased_item[user_id].extend(current_session)
                    # 随机挑选一个作为prediction item
                    item = random.choice(current_session)
                    self.train_items.append(item)
                    current_session.remove(item)
                    self.train_sessions.append(current_session)
                    self.records_number += 1
                    # test_sessions是train_data的最後一個購物籃(短期)
                    if j == size-2:
                        self.test_sessions.append(current_session)

                # 对test的数据集也要格式化，test中每个用户都只有一个current session
                # target改成最後一個購物籃，test_sessions是train_data的最後一個購物籃(短期)，test_pre_sessions是train_data的最後一個購物籃之前的所有購物籃(長期)
                self.test_users.append(user_id)
                current_session = [int(it) for it in sessions[size - 1].split(':')]
                self.test_real_items.append(current_session)
                self.test_pre_sessions.append(self.user_purchased_item[user_id])
        test_zipped = list(zip( self.test_users, self.test_sessions, self.test_pre_sessions, self.test_real_items))
        random.shuffle(test_zipped)
        self.test_users, self.test_sessions, self.test_pre_sessions, self.test_real_items = zip(*test_zipped)
        # test集中每个用户的预测的候选集就是整个item集合
        self.test_candidate_items = list(range(self.item_number))

        train_zipped = list(zip( self.train_users, self.train_sessions, self.train_pre_sessions, self.train_neg_items, self.train_items))
        random.shuffle(train_zipped)
        self.train_users, self.train_sessions, self.train_pre_sessions, self.train_neg_items, self.train_items = zip(*train_zipped)

    def gen_neg(self, user_id):
        neg_item_set = set()
        while len(neg_item_set) < self.neg_number:
            neg_item = np.random.randint(self.item_number)
            while neg_item in self.user_purchased_item[user_id]:
                neg_item = np.random.randint(self.item_number)
            neg_item_set.add(neg_item)
        return list(neg_item_set)

    def gen_train_batch_data(self, batch_size):
        l = len(self.train_users)

        if self.train_batch_id >= l:
            self.train_batch_id = 0

        batch_user = self.train_users[self.train_batch_id:self.train_batch_id + batch_size]
        batch_item = self.train_items[self.train_batch_id:self.train_batch_id + batch_size]
        batch_session = self.train_sessions[self.train_batch_id]
        if self.neg_number>1:
            batch_neg_item = self.train_neg_items[self.train_batch_id:self.train_batch_id + batch_size][0]
        else:
            batch_neg_item = self.train_neg_items[self.train_batch_id:self.train_batch_id + batch_size]
        batch_pre_session = self.train_pre_sessions[self.train_batch_id]

        self.train_batch_id = self.train_batch_id + batch_size

        return batch_user, batch_item, batch_session, batch_neg_item, batch_pre_session

    def gen_test_batch_data(self, user_id, batch_size):

        batch_user = self.test_users[user_id:user_id + batch_size]
        batch_item = self.test_candidate_items
        batch_session = self.test_sessions[user_id]
        batch_pre_session = self.test_pre_sessions[user_id]

        return batch_user, batch_item, batch_session, batch_pre_session

In [3]:
class shan():
    def __init__(self, data_type, global_dimension, epochs, lr, lamada_u_v, lamada_a, num):
        print('init ... ')
        self.input_data_type = data_type
        
        self.dg = data_generation(self.input_data_type)
        # 数据格式化
        self.dg.gen_train_test_data()

        self.train_user_purchased_item_dict = self.dg.user_purchased_item

        self.user_number = self.dg.user_number
        self.item_number = self.dg.item_number
        self.neg_number = self.dg.neg_number

        self.test_users = self.dg.test_users
        self.test_candidate_items = self.dg.test_candidate_items
        self.test_sessions = self.dg.test_sessions
        self.test_pre_sessions = self.dg.test_pre_sessions
        self.test_real_items = self.dg.test_real_items

        self.global_dimension = global_dimension # 論文: 100
        self.batch_size = 1
        self.K = 10
        self.results = []  # 可用来保存test每个用户的预测结果，最终计算precision
        self.num = num # 紀錄第幾次實驗用

        self.step = 0
        self.iteration = epochs
        self.lr = 0.05
        self.lamada_u_v = lamada_u_v # 論文: [0.01、0.001、0.0001]
        self.lamada_a = lamada_a  # 論文: [0,1,10,50]

        self.loss = 0
        self.path = f'{str(self.num)}_Shan_{self.input_data_type}_d{str(self.global_dimension)}_lr{str(self.lr)}_neg{str(self.neg_number)}_uv{str(self.lamada_u_v)}_a{str(self.lamada_a)}'
        
        # 日志基本配置
#         logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger()
#         fh = logging.FileHandler(f'./results/{self.input_data_type}/{self.path}' , mode='a', encoding=None, delay=False)
#         self.logger.addHandler(fh)

        self.initializer = tf.random_normal_initializer(mean=0, stddev=0.01)
        self.initializer_param = tf.random_uniform_initializer(minval=-np.sqrt(3 / self.global_dimension),
                                                               maxval=-np.sqrt(3 / self.global_dimension))

        self.user_id = tf.compat.v1.placeholder(tf.int32, shape=[None], name='user_id')
        self.item_id = tf.compat.v1.placeholder(tf.int32, shape=[None], name='item_id')
        # 不管是当前的session，还是之前的session集合，在数据处理阶段都是一个数组，数组内容为item的编号
        self.current_session = tf.compat.v1.placeholder(tf.int32, shape=[None], name='current_session')
        self.pre_sessions = tf.compat.v1.placeholder(tf.int32, shape=[None], name='pre_sessions')
        self.neg_item_id = tf.compat.v1.placeholder(tf.int32, shape=[None], name='neg_item_id')

        self.user_embedding_matrix = tf.compat.v1.get_variable('user_embedding_matrix', initializer=self.initializer,
                                                     shape=[self.user_number, self.global_dimension])
        self.item_embedding_matrix = tf.compat.v1.get_variable('item_embedding_matrix', initializer=self.initializer,
                                                     shape=[self.item_number, self.global_dimension])
        self.the_first_w = tf.compat.v1.get_variable('the_first_w', initializer=self.initializer_param,
                                           shape=[self.global_dimension, self.global_dimension])
        self.the_second_w = tf.compat.v1.get_variable('the_second_w', initializer=self.initializer_param,
                                            shape=[self.global_dimension, self.global_dimension])
        self.the_first_bias = tf.compat.v1.get_variable('the_first_bias', initializer=self.initializer_param,
                                              shape=[self.global_dimension])
        self.the_second_bias = tf.compat.v1.get_variable('the_second_bias', initializer=self.initializer_param,
                                               shape=[self.global_dimension])

    def attention_level_one(self, user_embedding, pre_sessions_embedding, the_first_w, the_first_bias):
        self.weight = tf.nn.softmax(tf.transpose(tf.matmul(
            tf.sigmoid( tf.add(tf.matmul(pre_sessions_embedding, the_first_w), the_first_bias))  # matmul((?,20)(20,20))=(?,20), add=(?,20), 
            , tf.transpose(user_embedding))))  #  matmul((?,20),(20, ?))=>(?,?)

        out = tf.reduce_sum(tf.multiply( pre_sessions_embedding, tf.transpose(self.weight)), axis=0)# (?, 20) (?,?) => (20,)
        return out

    def attention_level_two(self, long_user_embedding, current_session_embedding, the_second_w, the_second_bias):
        # 需要将long_user_embedding加入到current_session_embedding中来进行attention，
        # 论文中规定，long_user_embedding的表示也不会根据softmax计算得到的参数而变化。
        # self.weight=>(n,1)
        self.weight = tf.nn.softmax(tf.transpose(tf.matmul( tf.sigmoid(
            tf.add( tf.matmul(tf.concat([current_session_embedding, tf.expand_dims(long_user_embedding, axis=0)], 0), the_second_w), the_second_bias)),  # matmul((?,20)(20,20)), =>shape=(?,20)
                tf.transpose(tf.expand_dims(long_user_embedding, axis=0)) ))) #the_second_bias:shape=(20,) ,add(?,20) => matmul((?,20)(20, 1))

        out = tf.reduce_sum(
            tf.multiply(tf.concat([current_session_embedding, tf.expand_dims(long_user_embedding, axis=0)], 0), tf.transpose(self.weight)), axis=0)
        return out

    def build_model(self):
        print('building model ... ')
        self.user_embedding = tf.nn.embedding_lookup(self.user_embedding_matrix, self.user_id)
        self.item_embedding = tf.nn.embedding_lookup(self.item_embedding_matrix, self.item_id)
        self.current_session_embedding = tf.nn.embedding_lookup(self.item_embedding_matrix, self.current_session)
        self.pre_sessions_embedding = tf.nn.embedding_lookup(self.item_embedding_matrix, self.pre_sessions)
        self.neg_item_embedding = tf.nn.embedding_lookup(self.item_embedding_matrix, self.neg_item_id)

        self.long_user_embedding = self.attention_level_one(self.user_embedding, self.pre_sessions_embedding,
                                                            self.the_first_w, self.the_first_bias)
        self.hybrid_user_embedding = self.attention_level_two(self.long_user_embedding, self.current_session_embedding,
                                                              self.the_second_w, self.the_second_bias)
        
        # compute preference
        self.positive_element_wise = tf.matmul(tf.expand_dims(self.hybrid_user_embedding, axis=0),
                                               tf.transpose(self.item_embedding))
        self.negative_element_wise = tf.matmul(tf.expand_dims(self.hybrid_user_embedding, axis=0),
                                               tf.transpose(self.neg_item_embedding))
        
        self.intention_loss = tf.reduce_mean(
            -tf.math.log(tf.nn.sigmoid(self.positive_element_wise - self.negative_element_wise)))
        self.regular_loss_u_v = tf.add(self.lamada_u_v * tf.nn.l2_loss(self.user_embedding),
                                       self.lamada_u_v * tf.nn.l2_loss(self.item_embedding))
        self.regular_loss_a = tf.add(self.lamada_a * tf.nn.l2_loss(self.the_first_w),
                                     self.lamada_a * tf.nn.l2_loss(self.the_second_w))
        self.regular_loss = tf.add(self.regular_loss_a, self.regular_loss_u_v)
        self.intention_loss = tf.add(self.intention_loss, self.regular_loss)

        # 增加test操作，由于每个用户pre_sessions和current_session的长度不一样，
        # 所以无法使用同一个矩阵进行表示同时计算，因此每个user计算一次，将结果保留并进行统计
        # 注意，test集合的整个item_embeeding得到的是 [M*K]的矩阵，M为所有item的个数，K为维度
        self.top_value, self.top_index = tf.nn.top_k(self.positive_element_wise, k=self.K, sorted=True)


    def run(self):
        print('running ... ')
        with tf.compat.v1.Session() as self.sess:
            self.intention_optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=self.lr).minimize(
                self.intention_loss)
            init = tf.compat.v1.global_variables_initializer()
            self.sess.run(init)

            results = []
            for iter in range(1, self.iteration+1):
                print('new iteration begin ... ')
                print('iteration: ', str(iter))
                self.logger.info('Epoch: ' + str(iter)+"\n")
                all_loss = []
                while self.step * self.batch_size < self.dg.records_number:
                    # 按批次读取数据
                    batch_user, batch_item, batch_session, batch_neg_item, batch_pre_sessions = self.dg.gen_train_batch_data(
                        self.batch_size)
                    
                    _, loss = self.sess.run([self.intention_optimizer, self.intention_loss],
                                  feed_dict={self.user_id: batch_user,
                                             self.item_id: batch_item,
                                             self.current_session: batch_session,
                                             self.neg_item_id: batch_neg_item,
                                             self.pre_sessions: batch_pre_sessions
                                             })
                    all_loss.append(loss)
                
                    self.step += 1

                self.loss =  np.mean(all_loss)
                self.logger.info('loss' + ' = ' + str(self.loss))

                if np.isnan(self.loss):
                    self.logger.info('early stop...')
                    print("early stop...")
                    break

                self.step = 0
                result = self.evolution( iter )
                results.append(result)

                record_df = pd.DataFrame(results,columns=['Epoch','Recall@5', 'Recall@10', 'Recall@30', 'Recall@50', 'Recall@65',
                                                          'Precision@5', 'Precision@10', 'Precision@30', 'Precision@50', 'Precision@65',
                                                          'F1-score@5', 'F1-score@10', 'F1-score@30', 'F1-score@50', 'F1-score@65',
                                                          'NDCG@5', 'NDCG@10', 'NDCG@30', 'NDCG@50', 'NDCG@65',
                                                          'MAE@5', 'MAE@10', 'MAE@30', 'MAE@50', 'MAE@65', 'Loss'])
                if not os.path.exists("./output"):
                    os.mkdir("./output")
                path = f'./output/{self.path}.csv'
                record_df.to_csv(path, index=False)


    def precision_k(self, pre_top_k, true_items):
        right_pre = 0
        user_number = len(pre_top_k)
        for i in range(user_number):
            if true_items[i] in pre_top_k[i]:
                right_pre += 1
        return right_pre / user_number

    def evolution(self, iter):
        pre_top_k = []
        predictions = []

        for user_id in self.test_users:
            batch_user, batch_item, batch_session, batch_pre_session = self.dg.gen_test_batch_data(user_id,
                                                                                                   self.batch_size)
            top_k_value, top_index, prediction = self.sess.run([self.top_value, self.top_index, self.positive_element_wise],
                                                   feed_dict={self.user_id: batch_user,
                                                              self.item_id: batch_item,
                                                              self.current_session: batch_session,
                                                              self.pre_sessions: batch_pre_session})
            pre_top_k.append(top_index)
            predictions.append(prediction)

        k_list=[5,10,30,50,65]

        f1_score_at_k_eval, recall_at_k_eval, precision_at_k_eval = self.calculate_f1_score_at_k(predictions, self.test_real_items, k_list)
        
        recall_5_rec = recall_at_k_eval['Recall@5']
        recall_10_rec = recall_at_k_eval['Recall@10']
        recall_30_rec = recall_at_k_eval['Recall@30']
        recall_50_rec = recall_at_k_eval['Recall@50']
        recall_65_rec = recall_at_k_eval['Recall@65']
        recall_list = [recall_5_rec, recall_10_rec, recall_30_rec, recall_50_rec, recall_65_rec]
        
        precision_5_rec = precision_at_k_eval['Precision@5']
        precision_10_rec = precision_at_k_eval['Precision@10']
        precision_30_rec = precision_at_k_eval['Precision@30']
        precision_50_rec = precision_at_k_eval['Precision@50']
        precision_65_rec = precision_at_k_eval['Precision@65']
        precision_list = [precision_5_rec, precision_10_rec, precision_30_rec, precision_50_rec, precision_65_rec]
        
        f1_5_rec = f1_score_at_k_eval['F1-score@5']
        f1_10_rec = f1_score_at_k_eval['F1-score@10']
        f1_30_rec = f1_score_at_k_eval['F1-score@30']
        f1_50_rec = f1_score_at_k_eval['F1-score@50']
        f1_65_rec = f1_score_at_k_eval['F1-score@65']
        f1_list = [f1_5_rec, f1_10_rec, f1_30_rec, f1_50_rec, f1_65_rec]

        ndcg_at_k_eval = self.calculate_ndcg_at_k(predictions, self.test_real_items, k_list)
        ndcg_5_rec = ndcg_at_k_eval['NDCG@5']
        ndcg_10_rec = ndcg_at_k_eval['NDCG@10']
        ndcg_30_rec = ndcg_at_k_eval['NDCG@30']
        ndcg_50_rec = ndcg_at_k_eval['NDCG@50']
        ndcg_65_rec = ndcg_at_k_eval['NDCG@65']
        ndcg_list = [ndcg_5_rec, ndcg_10_rec, ndcg_30_rec, ndcg_50_rec, ndcg_65_rec]

        mae_at_k_eval = self.calculate_mae_at_k(self.test_real_items, k_list)
        mae_5_rec = mae_at_k_eval['MAE@5']
        mae_10_rec = mae_at_k_eval['MAE@10']
        mae_30_rec = mae_at_k_eval['MAE@30']
        mae_50_rec = mae_at_k_eval['MAE@50']
        mae_65_rec = mae_at_k_eval['MAE@65']
        mae_list = [mae_5_rec, mae_10_rec, mae_30_rec, mae_50_rec, mae_65_rec]

        result = [iter] + recall_list + precision_list + f1_list + ndcg_list + mae_list + [self.loss]
        print("calculate_recall_at_k=", recall_at_k_eval)
        print("calculate_precision_at_k=", precision_at_k_eval)
        print("calculate_f1_score_at_k=", f1_score_at_k_eval)
        print("calculate_ndcg_at_k=", ndcg_at_k_eval)
        print("calculate_mae_at_k=", mae_at_k_eval)

        return result

    def calculate_f1_score_at_k(self, predictions, labels_list, k_list):
        # 將預測機率矩陣轉換為 PyTorch 張量。
        predictions = torch.from_numpy(np.array(predictions, dtype=np.float32))#.to('cuda')
        num_users = len(labels_list)
        f1_score_at_k_eval = dict()
        recall_at_k_eval = dict()
        precision_at_k_eval = dict()
        for k in k_list:
            f1_score_sum = 0.0
            recall_sum = 0.0
            precision_sum = 0.0
            for i in range(num_users):
                    # 將用戶 i 的真實標籤轉換為 PyTorch 張量。
                    labels = torch.from_numpy(np.array(labels_list[i], dtype=np.int64))#.to('cuda')
                    # 計算用戶 i 在預測機率矩陣中機率最高的 K 個項目的索引。
                    top_k_item_labels = torch.topk(predictions[i], k)[1]
                    # 計算用戶 i 的真實標籤和預測標籤的交集。 # TP
                    true_positives = torch.sum(torch.sum(torch.eq(top_k_item_labels, labels.unsqueeze(1)).to(torch.float32), dim=1)).item()
                    # 計算用戶 i 的真實標籤和預測標籤的並集。
                    predicted_positives = k # TP+FP
                    actual_positives = len(labels) # TP+FN
                    if actual_positives == 0:
                        precision = 0.0
                        recall = 0.0
                    else:
                        precision = true_positives / predicted_positives
                        recall = true_positives / actual_positives
                    # 計算 F1-score。
                    if precision + recall == 0:
                        f1_score = 0.0
                    else:
                        f1_score = 2 * precision * recall / (precision + recall)
                    f1_score_sum += f1_score
                    recall_sum += recall
                    precision_sum += precision
            # 計算平均 F1-score@K 分數。
            f1_score_at_k = f1_score_sum / float(num_users)
            recall_at_k = recall_sum / float(num_users)
            precision_at_k = precision_sum / float(num_users)
            key = '{}@{}'.format('F1-score',k)
            f1_score_at_k_eval[key]=f1_score_at_k
            key = '{}@{}'.format('Recall',k)
            recall_at_k_eval[key]=recall_at_k
            key = '{}@{}'.format('Precision',k)
            precision_at_k_eval[key]=precision_at_k
        return f1_score_at_k_eval, recall_at_k_eval, precision_at_k_eval

    # NDCG@K
    def calculate_ndcg_at_k(self, predictions, labels_list, k_list):
        # 將預測機率矩陣轉換為 PyTorch 張量。
        predictions = torch.from_numpy(np.array(predictions, dtype=np.float32))
        num_users = len(labels_list)
        ndcg_at_k_eval = dict()
        for k in k_list:
            ndcg_sum = 0.0
            for i in range(num_users):
                # 將用戶 i 的真實標籤轉換為 PyTorch 張量。
                labels = torch.from_numpy(np.array(labels_list[i], dtype=np.int64))
                # 計算用戶 i 在預測機率矩陣中機率最高的 K 個項目的索引=標籤。
                top_k_item_labels = torch.topk(predictions[i], k)[1]
                # 計算 DCG@K。
                dcg_at_k = torch.sum(torch.div(1.0, torch.log2(torch.arange(k, dtype=torch.float32) + 2)) * (torch.eq(top_k_item_labels, labels.unsqueeze(1)).to(torch.float32) ))
                # 計算 IDCG@K。
                idcg_at_k = torch.sum(torch.div(1.0, torch.log2(torch.arange(len(labels), dtype=torch.float32) + 2)))
            
                # 計算 NDCG@K。
                ndcg_at_k = dcg_at_k / idcg_at_k
                ndcg_sum += ndcg_at_k.item()
            # 計算平均 NDCG@K 分數。
            ndcg_at_k = ndcg_sum / float(num_users)
            key = '{}@{}'.format('NDCG',k)
            ndcg_at_k_eval[key]=ndcg_at_k
        return ndcg_at_k_eval
    
    # MAE
    def calculate_mae_at_k(self, labels_list, k_list):
        num_users = len(labels_list)
        mae_eval = dict()
        for k in k_list:
            mae_sum = 0.0
            for i in range(num_users):
                labels = torch.from_numpy(np.array(labels_list[i], dtype=np.int64))
                mae_sum += abs(k - len(labels))
            key = "{}@{}".format("MAE", k)
            mae_eval[key] = mae_sum / float(num_users)

        return mae_eval

In [4]:
if __name__ == '__main__':
    type = 'Dunnhumby'   # TaFeng, Dunnhumby
    global_dimension = 100  # TaFeng: 50, Dunnhumby: 100
    epochs = 15  #跑幾個 epochs:  TaFeng: 15 、 Dnnhumby: 15
    lr = 0.001    # TaFeng: 0.05, Dunnhumby: 0.05
    lamada_u_v = 0.001 # 論文: [0.01、0.001、0.0001]
    lamada_a = 1  # 論文: [0,1,10,50]
    num = 1 # 紀錄第幾次實驗用，每輪需手動調
    model = shan(type, global_dimension, epochs, lr, lamada_u_v, lamada_a, num)
    model.build_model()
    model.run()

init ... 
init
user_number: 12826
item_number: 3003
building model ... 
running ... 
new iteration begin ... 
iteration:  1
calculate_recall_at_k= {'Recall@5': 0.08648893448036753, 'Recall@10': 0.11405744972811097, 'Recall@30': 0.16692517327432685, 'Recall@50': 0.20732729760616558, 'Recall@65': 0.23103587054144217}
calculate_precision_at_k= {'Precision@5': 0.11905504444098851, 'Precision@10': 0.08020427257134542, 'Precision@30': 0.04080253651437543, 'Precision@50': 0.030935599563387192, 'Precision@65': 0.02652304813540121}
calculate_f1_score_at_k= {'F1-score@5': 0.08215825509027244, 'F1-score@10': 0.07838876032604486, 'F1-score@30': 0.058092076180445686, 'F1-score@50': 0.04923714887264656, 'F1-score@65': 0.044155104975243116}
calculate_ndcg_at_k= {'NDCG@5': 0.10353190728802912, 'NDCG@10': 0.11976379474995894, 'NDCG@30': 0.14296585504554066, 'NDCG@50': 0.15691648481209536, 'NDCG@65': 0.16411660043715864}
calculate_mae_at_k= {'MAE@5': 5.3570092000623735, 'MAE@10': 6.437002962731951, 'MAE

calculate_recall_at_k= {'Recall@5': 0.05914181533292707, 'Recall@10': 0.08333316691989678, 'Recall@30': 0.13909503915218935, 'Recall@50': 0.1738466590895636, 'Recall@65': 0.19483424103578223}
calculate_precision_at_k= {'Precision@5': 0.08174021518790317, 'Precision@10': 0.05935599563387276, 'Precision@30': 0.03416497738967983, 'Precision@50': 0.025863090597224803, 'Precision@65': 0.022338039319173855}
calculate_f1_score_at_k= {'F1-score@5': 0.05637165856044427, 'F1-score@10': 0.05761499885740795, 'F1-score@30': 0.048611206919816935, 'F1-score@50': 0.041204105903992386, 'F1-score@65': 0.03721540473036242}
calculate_ndcg_at_k= {'NDCG@5': 0.0690555072683041, 'NDCG@10': 0.08327037371513801, 'NDCG@30': 0.10730035043846245, 'NDCG@50': 0.11915367053821392, 'NDCG@65': 0.1255756988855677}
calculate_mae_at_k= {'MAE@5': 5.3570092000623735, 'MAE@10': 6.437002962731951, 'MAE@30': 22.265476376111025, 'MAE@50': 42.127631373772026, 'MAE@65': 57.12498050834243}
new iteration begin ... 
iteration:  11
c